# Lab | Model Fitting and Evaluating.

## Activity (Tuesday)

## Linear Regression

## 1 X-y split (y is the target variable, in this case, "total claim amount")
## 2 Train-test split.
## 3 Standardize the data (after the data split!).
## 4 Apply linear regression.
## 5 Model Interpretation.

## Load the data 

In [242]:
import pandas as pd 
import seaborn as sns

In [243]:
csv_path = '/Users/matthewbatchelor/Downloads/marketing_customer_analysis_clean.csv'

In [244]:
df = pd.read_csv(csv_path)

In [245]:
df.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


In [246]:
# Isolate numerical variables 

numericals_df = df.select_dtypes(include=['number'])

In [247]:
numericals_df.head()

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1


In [248]:
numericals_df = numericals_df.dropna()

In [249]:
numericals_df = numericals_df.drop_duplicates()

## 1 X-y split (y is the target variable, in this case, "total claim amount")

In [250]:
X = numericals_df.drop(["total_claim_amount"], axis=1) # taking income as independent variable
y = numericals_df[["total_claim_amount"]]

## 2 Train-test split

In [251]:
from sklearn.model_selection import train_test_split

In [252]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

## 3 Standardize the data 

In [253]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

In [254]:
scaler = PowerTransformer()

In [255]:
scaler.fit(X_train)

PowerTransformer()

In [256]:
scaler.transform(X_train)

array([[-0.13287732,  1.71825926,  0.43773226, ..., -0.57987317,
        -0.06206636,  1.07649387],
       [ 1.01340122,  0.52005348,  0.36702354, ..., -0.57987317,
         0.51680123, -0.92894166],
       [-0.12106768, -1.81132298, -1.63587128, ..., -0.57987317,
        -1.1467141 ,  1.07649387],
       ...,
       [ 0.07277599,  0.44656524, -1.63587128, ..., -0.57987317,
        -0.06206636,  1.07649387],
       [-1.57896735, -0.27337082, -1.63587128, ..., -0.57987317,
         0.51680123,  1.07649387],
       [ 0.6111004 ,  2.21656148, -1.63587128, ..., -0.57987317,
        -0.06206636,  1.07649387]])

In [257]:
scaler.transform(X_test)

array([[-0.09783828, -0.02014458,  1.07885525, ..., -0.57987317,
         0.51680123,  1.07649387],
       [-0.02053214, -1.30159839,  0.3579988 , ...,  1.24086041,
        -1.1467141 , -0.92894166],
       [-0.0418935 , -1.92155253, -1.63587128, ...,  1.89908674,
        -1.1467141 ,  1.07649387],
       ...,
       [ 0.96272542, -1.06080836,  1.02445904, ..., -0.57987317,
        -1.1467141 , -0.92894166],
       [-1.96562504,  0.4931529 , -1.63587128, ..., -0.57987317,
        -0.06206636, -0.92894166],
       [-0.52398319, -0.21435533,  0.80030319, ..., -0.57987317,
         0.51680123, -0.92894166]])

## 4 Apply linear regression.

In [258]:
from sklearn.linear_model import LinearRegression

In [259]:
lm = LinearRegression()
model = lm.fit(X_train,y_train)

In [260]:
model.coef_

array([[ 5.51923191e-04, -6.21698962e-04, -3.29327123e-03,
         5.39064866e+00,  1.77468285e-01, -1.79264731e-01,
        -4.93935428e-01,  2.91981136e-01, -1.84174558e+00]])

In [261]:
model.intercept_

array([64.87810575])

## 5 Model Interpretation.

In [262]:
# Predict total claim amount based on income 

In [263]:
random_customer = X_test.sample()

In [264]:
random_customer

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,month
1904,1904,8826.611996,26320,111,3.0,92,0.0,9,2


In [265]:
model.predict(random_customer)

array([[555.10896258]])

## Create predictions for test set

In [266]:
y_pred = model.predict(X_test)

In [267]:
y_pred = pd.DataFrame(y_pred)

In [268]:
y_test = y_test.reset_index(drop=True)

In [269]:
resiudals_df = pd.concat([y_test,y_pred],axis=1)

In [270]:
residuals_df = resiudals_df.rename(columns={"total_claim_amount":"y_test", 0:"y_pred"})

In [271]:
residuals_df["residual"] = residuals_df["y_test"]-residuals_df["y_pred"]

In [272]:
residuals_df.head()

,y_test,y_pred,residual
0,475.423848,122.658353,352.765495
1,350.400000,350.524852,-0.124852
2,482.400000,415.505553,66.894447
3,673.342650,433.532813,239.809837
4,302.400000,154.757902,147.642098


## Model interpretation 

In [273]:
mean_error = residuals_df["residual"].mean()

In [274]:
mean_error

4.673648918980246

In [275]:
from sklearn.metrics import mean_squared_error as mse , mean_absolute_error as mae

In [276]:
mse(y_test,y_pred)

40504.74987548373

In [277]:
mae(y_test,y_pred)

146.03572323631195

In [278]:
rmse = mse(y_test,y_pred, squared = False)

In [279]:
rmse

201.25791878950685